# Synthesize a coloring book

In [1]:
import os
import numpy as np
import skimage
from PIL import Image
from PIL.Image import Image as PilImage
import json

from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoImageProcessor, Swin2SRForImageSuperResolution
import torch
from torch import inf

import openai

# !pip install python-docx 
from docx import Document
from docx.shared import Inches, Mm
import re

from transformers.testing_utils import torch_device

2023-05-12 17:19:06.555496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Inputs

In [2]:
images_to_generate = 16
# upscale image
enhance_images = True
image_width = 768
image_height = 1024
image_dir = "./images"

## Define keys and load models

In [3]:
 # Azure Instance
openai.api_key = ""
# your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_base =  "" 
openai.api_version = '2022-12-01' # this may change in the future
openai.api_type = 'azure'
#This will correspond to the custom name you chose for your deployment when you deployed a model. 
deployment_name='' 

def load_prompter():
    prompter_model = AutoModelForCausalLM.from_pretrained("microsoft/Promptist")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    return prompter_model, tokenizer

prompter_model, prompter_tokenizer = load_prompter()

def load_upscaler():
    model_name = 'caidas/swin2SR-classical-sr-x2-64'
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = Swin2SRForImageSuperResolution.from_pretrained(model_name).to(torch_device)
    return model, processor
    
upscale_model, upscale_processor = load_upscaler()

#model_id = "dreamlike-art/dreamlike-photoreal-2.0"
#model_id = "darkstorm2150/Protogen_Infinity_Official_Release"
model_id = "LottePeisch/RevAnimated-Diffusers"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, safety_checker = None)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

/home/gensyn/anaconda3/lib/python3.10/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/home/gensyn/anaconda3/lib/python3.10/site-packages/safetensors/torch.py:98: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/gensyn/anaconda3/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly,

## Helper functions

In [22]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()
    return data

# create a sketch for coloring based in an input image
def generate_coloring_page(input: PilImage) -> PilImage:
    # Convert to grayscale if needed
    if input.mode != "L":
        input = input.convert("L")
    np_image = np.asarray(input)
    
    # detect edges
    np_image = skimage.filters.sobel(np_image)
    # convert to 8 bpp
    np_image = skimage.util.img_as_ubyte(np_image)
    # Invert to get dark edges on a light background
    np_image = 255 - np_image
    # Improve the contrast
    np_image = skimage.exposure.rescale_intensity(np_image)
    
    return Image.fromarray(np_image)

# Create ideas for each desired page of the coloring book
def generate_coloring_book_page_descriptions(topic, number_pages):
    prompt = "Create "+str(number_pages)+" unique ideas for a coloring book on the topic of "+topic+"/n/nUnique ideas for coloring book topic:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result

# Create titles from story idea
def generate_title(idea):
    prompt = "Create a title for this story idea: "+idea
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=500
        )
    result = response.choices[0].text.strip()
    return result

# Create fabe fro idea and title
def generate_fable(idea, title):
    prompt = "Create a best selling fable with the title: "+title+"\n\nA plot summary is: "+idea+"\n\nCreate about a 2 page best selling fable, with the moral articulated at the end.  Fable:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=500
        )
    result = response.choices[0].text.strip()
    return result


# Create fun facts about the image
def generate_fun_facts(topic):
    prompt = "Create fun facts about "+topic+"/n/nFun facts:"
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.7,
            max_tokens=1500
        )
    result = remove_after_create(response.choices[0].text.strip())
    return result

# Create fun facts about the image
def generate_summary(prompt: str):
    response = openai.Completion.create(
            engine=deployment_name,
            prompt=prompt,
            temperature=0.5,
            max_tokens=1500
        )
    result = response.choices[0].text.strip()
    return result

# parse a string to list based on return characters
def parse_string(input_string: str) -> list:
    result = []
    lines = input_string.split('\n')
    for line in lines:
        if not 'Create ' in line and not line == "":
            result.append(remove_number_period(line.strip()))
    return result

def remove_after_create(s: str) -> str:
    keyword = "Create "
    index = s.find(keyword)
    
    if index == -1:
        # If "Create " is not found, return the original string
        return s
    
    # Find the end index of the keyword
    end_index = index + len(keyword)
    
    # Return the string up to the end of the keyword
    return s[:end_index]

def remove_number_period(s: str) -> str:
    # Use a regular expression to match a number followed by a period and a space
    pattern = r'^\d+\.\s'
    
    # Remove the matched pattern and return the modified string
    return re.sub(pattern, '', s).strip()

def generate(plain_text: str) -> str:
    input_ids = prompter_tokenizer(plain_text.strip()+" Rephrase:", return_tensors="pt").input_ids
    eos_id = prompter_tokenizer.eos_token_id
    outputs = prompter_model.generate(input_ids, do_sample=False, max_new_tokens=75, num_beams=8, num_return_sequences=8, eos_token_id=eos_id, pad_token_id=eos_id, length_penalty=-1.0)
    output_texts = prompter_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    res = output_texts[0].replace(plain_text+" Rephrase:", "").strip()
    return res

def enhance_image(image):
    # prepare image for the model
    inputs = upscale_processor(image, return_tensors="pt").to(torch_device)

    # forward pass
    with torch.no_grad():
        outputs = upscale_model(**inputs)

    # postprocess
    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    
    return Image.fromarray(output)

def create_directory_if_not_exists(path:str):
    """
    Creates a directory at the specified path if it does not already exist.
    """
    if not os.path.exists(path):
        os.makedirs(path)
    return

def load_file_lines(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]  # Remove trailing newline characters and leading/trailing whitespace
    return lines

def load_file_items(filename):
    with open(filename, 'r') as file:
        content = file.read()
    items = content.split(',')  # Split the content using commas as separators
    items = [item.strip() for item in items]  # Remove leading/trailing whitespace from each item
    return items

def write_list_to_file(filename, lines):
    with open(filename, 'w') as file:
        for line in lines:
            file.write(line + '\n')
            
def store_list_as_jsonl(filename, lines):
    with open(filename, 'w') as file:
        for line in lines:
            json.dump(line, file)
            file.write('\n')
            
def read_jsonl_as_list(filename):
    lines = []
    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line:
                lines.append(json.loads(line))
    return lines

## Workflow

In [23]:
#Read in ideas from file ideas.txt
ideas = read_file_to_list("ideas.txt")
ideas

['"The Day the Sun Slept In": An adventure where children around the world must wake up the sun when it oversleeps.',
 '"Elvin the Miniature Elephant": The journey of a small elephant who uses his size to do big things.',
 '"Haley\'s Hiccups": A magical story about a girl whose hiccups make crazy things happen.',
 '"The Boy Who Drew the Future": A story about a boy whose drawings come to life.',
 '"Sammy\'s Magic Glasses": A tale of a boy who sees hidden magical creatures through his glasses.',
 '"Planet of the Candy Aliens": The adventure of children discovering a planet full of sweet, living candies.',
 '"The Stargazer\'s Map": A tale about a map that can navigate through constellations.',
 '"Flora\'s Flower": A story about a magical flower that grants wishes.',
 '"The Tale of the Talking Turtle": A story about a wise old turtle that can talk.',
 '"Molly and the Moon Moth": A girl befriends a magical moon moth that takes her to different dimensions.',
 '"The Invisible Island": An isl

In [26]:
titles = []
for idea in ideas:
    title = generate_title(idea)
    titles.append(title)

In [27]:
write_list_to_file("titles.txt", titles)

In [28]:
#titles = load_file_items("titles.txt")

In [29]:
titles

['"A Worldwide Wake-Up Call: The Day the Sun Slept In"',
 '"Elvin the Miniature Elephant: Achieving Big Things in a Small Package"',
 '"Haley\'s Hiccup Adventures: A Magical Tale of Mischief and Mayhem"',
 '"The Dreamer Who Painted Prophecy"',
 '"Sammy\'s Spectacular Sight: A Journey of Discovery through Magical Glasses"',
 '"A Sweet Discovery: Exploring the Planet of the Candy Aliens"',
 '"The Stargazer\'s Guide to the Heavens: An Epic Journey with the Constellations"',
 '"Flora and the Wishing Flower"',
 '"The Wisdom of the Talking Turtle"',
 '"Molly and the Magic of the Moon Moth"',
 '"The Magic of Childhood: Discovering the Invisible Island"',
 '"Benji and the Chrono-Chronometer: A Time Travelling Adventure"',
 '"The Wisdom of the Whispering Willows"',
 '"The Magic of the Rainbow River"',
 '"Liam and the Leprechaun: A Quest for Gold"',
 '"Stepping Out: The Tale of the Dancing Penguin\'s Dream"',
 '"The Magic of Amelia\'s Cloud Castle"',
 '"The Incredible Discovery of a Tiny Dinosau

In [30]:
ideas_title = list(zip(ideas, titles))
ideas_title

[('"The Day the Sun Slept In": An adventure where children around the world must wake up the sun when it oversleeps.',
  '"A Worldwide Wake-Up Call: The Day the Sun Slept In"'),
 ('"Elvin the Miniature Elephant": The journey of a small elephant who uses his size to do big things.',
  '"Elvin the Miniature Elephant: Achieving Big Things in a Small Package"'),
 ('"Haley\'s Hiccups": A magical story about a girl whose hiccups make crazy things happen.',
  '"Haley\'s Hiccup Adventures: A Magical Tale of Mischief and Mayhem"'),
 ('"The Boy Who Drew the Future": A story about a boy whose drawings come to life.',
  '"The Dreamer Who Painted Prophecy"'),
 ('"Sammy\'s Magic Glasses": A tale of a boy who sees hidden magical creatures through his glasses.',
  '"Sammy\'s Spectacular Sight: A Journey of Discovery through Magical Glasses"'),
 ('"Planet of the Candy Aliens": The adventure of children discovering a planet full of sweet, living candies.',
  '"A Sweet Discovery: Exploring the Planet of 

In [31]:
fable = generate_fable(ideas_title[0][0], ideas_title[0][1])

In [32]:
stories = []
for item in ideas_title:
    fable = generate_fable(item[0], item[1])
    stories.append(fable)
    
store_list_as_jsonl("fables.jsonl", stories)

In [20]:
#stories = load_file_items("fables.txt")

In [33]:
topic = "365 children's stories of fables about fantasy, science fiction, adventure, magic, mystery, mythical, etc."

In [34]:
# create a summary
prompt = "Create a summary for a book about "+topic
summary = generate_summary(prompt)
summary

"This book is a collection of 365 children's stories about fantasy, science fiction, adventure, magic, mystery, mythical creatures, and more! Each story is unique and captivating, and will take readers on a journey of imagination. With stories about brave knights, magical creatures, and faraway lands, this book is sure to provide hours of entertainment and delight for readers of all ages."

In [35]:
len(stories)

379

In [ ]:
# create fun facts and images for each page
resume_after = 0
images = []
count = 0
negative_prompt = "watermark, text, error, blurry, jpeg artifacts, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, username, artist name, bad anatomy, extra limbs, extra body parts, floating body parts, extra legs, extra hands, missing fingers, bad fingers"
create_directory_if_not_exists(image_dir)
# iterate for each desired page
for story in stories:
    if count < resume_after:
        count = count + 1
        continue
    prompt = "award winning artwork to illustrate a fable, "+ideas[count]+", beautiful, imaginative, intricate details, centered"
    # improve the image prompt with a model
    new_prompt = generate(prompt)
    # iterate for the desired number of images to choose from for each page
    for image_num in range(0, images_to_generate):
        # create an image based on the improved prompt
        image = pipe(new_prompt, guidance_scale=8, num_inference_steps=80, height=image_height, width=image_width).images[0]
        # enhance/upscale the generated image
        if enhance_images:
            enhanced_image = enhance_image(image)
            # create a coloring book image based on the generated image
            coloring_book_image = generate_coloring_page(enhanced_image)
             # save the image with the format image_pageNumber_imageNumber as jpg
            coloring_book_image.save(image_dir+"/image_"+str(count)+"_"+str(image_num)+".jpg")
            enhanced_image.save(image_dir+"/photo_"+str(count)+"_"+str(image_num)+".jpg")

        else:
            coloring_book_image = generate_coloring_page(image)
            # save the image with the format image_pageNumber_imageNumber as jpg
            coloring_book_image.save(image_dir+"/image_"+str(count)+"_"+str(image_num)+".jpg")
            image.save(image_dir+"/photo_"+str(count)+"_"+str(image_num)+".jpg")
    count = count + 1

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
# write to a word document
# create a new docx document object
document = Document()

# set the document size and margins
section = document.sections[0]
section.page_width = Inches(8.5)
section.page_height = Inches(11.0)
section.left_margin = Inches(1.0)
section.right_margin = Inches(1.0)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)

# Add a title page
document.add_heading(title, 0)
p = document.add_paragraph('A coloring book about '+topic)
p = document.add_paragraph(summary)

count = 0
for fable in stories:
    document.add_page_break()
    document.add_heading(titles[count], level=1)
    document.add_picture(image_dir+"/image_"+str(count)+"_0.jpg", width=Inches(6.5))
    document.add_page_break()
    document.add_paragraph("Fable: "+titles[count], style='Intense Quote')
    document.add_paragraph(fable)
    count += 1

document.save('fables_coloring.docx')

# write to a word document
# create a new docx document object
document = Document()

# set the document size and margins
section = document.sections[0]
section.page_width = Inches(6.0)
section.page_height = Inches(9.0)
section.left_margin = Inches(0.375)
section.right_margin = Inches(0.375)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)

# Add a title page
document.add_heading(title, 0)
p = document.add_paragraph('An illustrated book about '+topic)
p = document.add_paragraph(summary)

count = 0
for fable in stories:
    document.add_page_break()
    document.add_heading(titles[count], level=1)
    document.add_picture(image_dir+"/photo_"+str(count)+"_0.jpg", width=Inches(5.25))
    document.add_page_break()
    document.add_paragraph('Fable: '+titles[count], style='Intense Quote')
    document.add_paragraph(fable)
    count += 1

document.save('fables_illustrated.docx')

In [10]:
!explorer.exe .